#Authoring VOEvent XML packets with ``voevent-parse``#

In [ ]:
from __future__ import print_function
import voeventparse as vp
import datetime

We'll start by creating the skeleton of our VOEvent packet. We set the role to test so that nobody is tempted to start acting on the contents of this demo event. We also set the timestamp in the Who block to the time the event was generated (not when the observation was made), as per the specification:

In [ ]:
v = vp.Voevent(stream='hotwired.org/gaia_demo', stream_id=1,
                       role=vp.definitions.roles.test)


In [ ]:
#Set *packet* timestamp to date of packet-generation:
vp.set_who(v, date=datetime.datetime.utcnow(), 
        author_ivorn="foo.hotwired.hotwireduniverse.org/bar")

vp.set_author(v, title="Hotwired VOEvent Hands-on",
                      contactName="Joe Bloggs")
v.Description = "This is not an official Gaia data product."

At any time, you can use ``vp.dumps`` (dump-string) to take a look at the VOEvent you've composed so far:

In [ ]:
# print(vp.dumps(v, pretty_print=True))

However, that's pretty dense! Use ``vp.prettystr`` to view a single element, which is a bit easier on the eyes:

In [ ]:
print(vp.prettystr(v.Who))

Now we need to specify where and when the observation was made. Rather than trying to specify a position for Gaia, we'll just call it out by name. Note that Gaia don't provide errors on the position they cite, so we're rather optimistically using 0:

In [ ]:
vp.add_where_when(v,
               coords=vp.Position2D(ra=168.47841, dec=-23.01221, err=0, units='deg',
                                    system=vp.definitions.sky_coord_system.fk5),
               obs_time=datetime.datetime(2014, 11, 7, 1, 5, 9),
               observatory_location="Gaia")

In [ ]:
#See how much element creation that just saved us...
print(vp.prettystr(v.WhereWhen))

In [ ]:
print([(c.tag, type(c)) for c in v.Who.getchildren()])

##Advanced##
Note that if you want to do something that's not part of the standard use-cases addressed by voevent-parse,
you can always use the underlying lxml.objectify tools to manipulate elements yourself.
For example - don't like the 'voevent-parse' tag that gets added to your VOEvent Who skeleton? You can delete it:

In [ ]:
v.Who.Description

In [ ]:
del v.Who.Description

In [ ]:
print(vp.prettystr(v.Who))

Want to add some additional elements? You can, but: make sure you stick to the VOEvent schema:

In [ ]:
import lxml.objectify as objectify

In [ ]:
vp.valid_as_v2_0(v)

In [ ]:
#If you just want a text-value element, you can simply assign to it:
v.What.foo1='bar1'

In [ ]:
# If you want an element that is part of a nested structure,
# but doesn't have a value in itself, you can use SubElement:
objectify.SubElement(v.What, 'foo2')
v.What.foo2.bar2='nestedbar2'

In [ ]:
print(vp.prettystr(v.What))

In [ ]:
# Obviously, these are non-schema compliant elements:
# (Use Params for storing general data.)
vp.valid_as_v2_0(v)